<a href="https://colab.research.google.com/github/deepakmoud/Signature-Verification/blob/master/Signature_Recognition_densenet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the Keras libraries and packages
import tensorflow as tf
from tensorflow.python import keras
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications import Xception
from keras.applications import VGG19
from keras.applications import ResNet152V2
from keras.applications import InceptionV3
from keras.applications import DenseNet121
import warnings 
warnings.filterwarnings('ignore')
img_width, img_height = 224, 224

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
# Input parameter
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train'
val_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test'
TEST_DIR='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/TEST1/'
filepath = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/vgg16_GDPS_xfer_weights.hdf5'
nb_epochs = 50
SIGNATURE_CLASSES = ['001', '002', '003','004','006','009','012','013','014','015','016','017','018','019','020','021','022','023','024','025','026','027','028','029','030','031','032','033','034','035','036','037','038','039','040','041','042','043','044','045','046','047','048','049','050','051','052','053','054','055','056','057','058','059','060','061','062','063','064','065','066','067','068','069']
print("Input parameters are assigned")

Input parameters are assigned


In [5]:
# image data generation
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train/'
val_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test/'
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=True, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=True,class_mode='categorical')

Found 672 images belonging to 64 classes.
Found 224 images belonging to 64 classes.


In [6]:
#Load the pretrained Network
denseNet121_model = DenseNet121( 
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(img_height,img_width,3),
    pooling=None,
    classes=1000
    
)
print("pretrained Network is loaded")

29089792/29084464 [==============================] - 1s 0us/step
pretrained Network is loaded


In [7]:
denseNet121_model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
________________________________________________________________________________________

In [8]:
# Freeze the layers
for layer in denseNet121_model.layers:
    layer.trainable = False
print("Pretrained layers are freezed")

Pretrained layers are freezed


In [9]:
# Model Creation
model = Sequential()
model.add(denseNet121_model)
#add fully connected layer:
input_layer=model.add(Flatten())
hidden_layer=model.add(Dense(128, activation='relu', name='hidden_layer'))
classification_layer=model.add(Dense(64, activation='relu', name='classification_layer'))
output_layer=model.add(Dense(64, activation='softmax', name='output_layer')) 
print("All layers top of pretrained layers are developed")

All layers top of pretrained layers are developed


In [10]:
# Model compilation
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [11]:
# model weights saving after each epoch
checkpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
print('Check point is created')

Check point is created


In [12]:
history = model.fit_generator( train_generator, callbacks = callbacks_list, nb_epoch=nb_epochs, validation_data=validation_generator)
print('Training Completed!')

Epoch 1/50
21/21 [==============================] - 182s 9s/step - loss: 4.6654 - accuracy: 0.0521 - val_loss: 4.0306 - val_accuracy: 0.0402
Epoch 2/50
21/21 [==============================] - 11s 536ms/step - loss: 3.5350 - accuracy: 0.1815 - val_loss: 4.2351 - val_accuracy: 0.0536
Epoch 3/50
21/21 [==============================] - 15s 700ms/step - loss: 2.9167 - accuracy: 0.2842 - val_loss: 4.3850 - val_accuracy: 0.0759
Epoch 4/50
21/21 [==============================] - 15s 699ms/step - loss: 2.4901 - accuracy: 0.3467 - val_loss: 3.4765 - val_accuracy: 0.1473
Epoch 5/50
21/21 [==============================] - 15s 702ms/step - loss: 1.9694 - accuracy: 0.4479 - val_loss: 4.3118 - val_accuracy: 0.1339
Epoch 6/50
21/21 [==============================] - 15s 705ms/step - loss: 1.3705 - accuracy: 0.6101 - val_loss: 4.2119 - val_accuracy: 0.1696
Epoch 7/50
21/21 [==============================] - 15s 703ms/step - loss: 0.9102 - accuracy: 0.7411 - val_loss: 4.6371 - val_accuracy: 0.1518
E

In [13]:
# save model and architecture to single file
model.save('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_idenseNet121_model.h5')
model.summary()

print("Saved model to disk")

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               6422656   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 64)                4160      
Total params: 13,472,576
Trainable params: 6,435,072
Non-trainable params: 7,037,504
_________________________________________________________________
Saved model to disk


In [14]:
# Loading saved model from Drive.
from keras.models import load_model
model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_idenseNet121_model.h5')
print("Model is Loaded")
model.summary()

Model is Loaded
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               6422656   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 64)                4160      
Total params: 13,472,576
Trainable params: 6,435,072
Non-trainable params: 7,037,504
_________________________________________________________________


In [0]:
import os
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train/'
def get_images(fish):
    """Load files from train folder"""
    fish_dir = train_data_dir+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

In [17]:
pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 105kB/s 
ERROR: umap-learn 0.4.2 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [0]:
def read_image(src):
    import os
    from scipy import misc
    filepath=src
    im=misc.imread(filepath)
    import scipy.misc  as mc
     
    return mc.imresize(im,(img_width, img_height))

In [19]:
files = []
y_all = []
for fish in SIGNATURE_CLASSES:
    fish_files = get_images(fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)
print(len(files))
print(len(y_all))


19 photos of 001
19 photos of 002
18 photos of 003
18 photos of 004
18 photos of 006
18 photos of 009
18 photos of 012
9 photos of 013
18 photos of 014
18 photos of 015
18 photos of 016
9 photos of 017
9 photos of 018
9 photos of 019
9 photos of 020
9 photos of 021
9 photos of 022
9 photos of 023
9 photos of 024
9 photos of 025
9 photos of 026
9 photos of 027
9 photos of 028
9 photos of 029
9 photos of 030
9 photos of 031
9 photos of 032
9 photos of 033
9 photos of 034
9 photos of 035
9 photos of 036
9 photos of 037
9 photos of 038
9 photos of 039
9 photos of 040
9 photos of 041
9 photos of 042
9 photos of 043
9 photos of 044
9 photos of 045
9 photos of 046
9 photos of 047
6 photos of 048
9 photos of 049
9 photos of 050
9 photos of 051
9 photos of 052
9 photos of 053
9 photos of 054
9 photos of 055
9 photos of 056
9 photos of 057
9 photos of 058
9 photos of 059
9 photos of 060
9 photos of 061
9 photos of 062
9 photos of 063
9 photos of 064
9 photos of 065
9 photos of 066
12 photos of 0

In [20]:
X_all = np.ndarray((len(files),img_width, img_height , 3), dtype=np.uint8)
print(X_all.shape)
for i, im in enumerate(files): 
    X_all[i] = read_image(train_data_dir+im)
    print(X_all[i])   
   
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))


print(X_all.shape)

Streaming output truncated to the last 5000 lines.
[[[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 252]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 ...

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]]
[[[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 ...

 

In [21]:
import os
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), img_width, img_height, 3), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    
test_preds = model.predict(test, verbose=1)
submission = pd.DataFrame(test_preds, columns=SIGNATURE_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

submission.to_csv('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureResultsdensenet.csv',index=False)

12/12 [==============================] - 5s 382ms/step


In [22]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
from keras.models import load_model
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Click to know Name " )
output = widgets.Output()

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_idenseNet121_model.h5')
img_path = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/11_052.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)
print(block4_pool_features)
label_index=block4_pool_features.argmax()
print(label_index)
def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    print("Signature is of  "  +SIGNATURE_CLASSES[label_index])

button.on_click(on_button_clicked)
display(button, output)

[[5.76039273e-20 4.38061940e-13 1.87226617e-21 1.15592613e-31
  4.55037453e-24 4.20874099e-11 4.79816504e-17 1.82614826e-06
  5.01925387e-17 2.40863182e-17 3.84496180e-17 9.52496472e-20
  1.27792298e-16 4.64793246e-25 1.63363507e-15 5.43208822e-21
  6.62592981e-07 6.98835391e-12 9.83088196e-20 1.01181626e-14
  1.00858241e-23 1.16966411e-11 2.47373730e-08 8.15725872e-12
  1.17140441e-24 4.34007570e-02 3.97507061e-18 2.97739505e-18
  1.53799277e-15 1.23318857e-12 1.35785534e-13 4.66852295e-12
  1.24571672e-10 3.24141270e-19 5.70404234e-20 2.49574426e-21
  5.15626008e-20 9.96660722e-17 3.00489363e-18 1.43420356e-12
  6.61153924e-18 9.18362469e-18 1.00280322e-26 2.05497251e-21
  6.03003365e-14 1.16313778e-13 6.12928861e-27 8.30015981e-19
  1.35569325e-11 1.12931615e-10 1.63162799e-17 3.59331590e-25
  1.31621707e-16 1.51076336e-13 2.58078721e-15 4.43092385e-27
  3.11553905e-17 9.61850822e-28 7.00655571e-13 9.44884181e-01
  8.62903027e-10 5.27780842e-14 4.27920726e-21 1.17125455e-02]]
59


Button(description='Click to know Name ', style=ButtonStyle())

Output()

In [23]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_inceptionV3_model.h5')
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(validation_generator)
print('test loss, test acc:', results)


# Evaluate on test data
7/7 [==============================] - 6s 857ms/step
test loss, test acc: [11.6864595413208, 0.09375]


In [24]:
# Extracting Features from classification Layer
from keras.models import Model
layer_name= 'classification_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
print("Imtermediate model is constructed")


Imtermediate model is constructed


In [25]:
# Compilation of intermediate model
intermediate_layer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [26]:
# Saving intermediate model
intermediate_layer_model.save('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_idensenetfolder_Intermediatemodel.h5')
intermediate_layer_model.summary()

print("Saved Intermediate model to disk")

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3_input (InputLay (None, 224, 224, 3)       0         
_________________________________________________________________
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               6553728   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 28,364,768
Trainable params: 6,561,984
Non-trainable params: 21,802,784
_________________________________________________________________
Saved Intermediate model to disk


In [27]:
# Loading Intermediate Model
from keras.models import load_model

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_idensenetfolder_Intermediatemodel.h5')
print("Intermediate model is loaded")

Intermediate model is loaded


In [28]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3_input (InputLay (None, 224, 224, 3)       0         
_________________________________________________________________
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 51200)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               6553728   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 28,364,768
Trainable params: 6,561,984
Non-trainable params: 21,802,784
_________________________________________________________________


In [29]:
# Training Label feature identification(y_train)
import numpy as np
batch_size=32
sample_count=672
features = np.zeros(shape=(672, 64))  # Must be equal to the output of the convolutional base
labels = np.zeros(shape=(672, 64))
i = 0
for inputs_batch, labels_batch in train_generator:
  features_batch = model.predict(inputs_batch)
  features[i * batch_size: (i + 1) * batch_size] = features_batch
  labels[i * batch_size: (i + 1) * batch_size] = labels_batch
  i += 1
  if i*batch_size  >= sample_count:
    break
print(labels.shape)
print(features.shape)


(672, 64)
(672, 64)


In [30]:
 print(labels)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [31]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=labels
print(labels_train.shape)
print(labels)
#print(features_train)


(672, 64)
(672, 64)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [32]:
# identification of test labels
import numpy as np
batch_size=32
sample_count=224
features_test = np.zeros(shape=(224, 64))  # Must be equal to the output of the convolutional base
labels_test = np.zeros(shape=(224,64))
i = 0
for inputs_batch, labels_batch in validation_generator:
  features_batch = model.predict(inputs_batch)
  features_test[i * batch_size: (i + 1) * batch_size] = features_batch
  labels_test[i * batch_size: (i + 1) * batch_size] = labels_batch
  i += 1
  if i*batch_size  >= sample_count:
    break
print(labels_test.shape)
print(features_test.shape)
print(labels_test)

(224, 64)
(224, 64)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [33]:
# #identification of testing Labels
print(features_test.shape)
#labels_test=np.expand_dims(labels_test, axis=1)
print(labels_test.shape)
print(features_test)

(224, 64)
(224, 64)
[[ 9.10485649  0.          0.         ...  0.          0.
   0.        ]
 [ 0.44588     3.47220373  0.         ... 10.29838371  0.
   0.        ]
 [ 0.          0.          0.         ...  0.88590825  0.
   0.        ]
 ...
 [ 0.          0.          0.         ... 21.9365654   0.
   0.        ]
 [17.23563004  0.          0.         ... 12.48611641  0.
   0.        ]
 [ 1.54980838  0.          0.         ...  2.22156262  0.
   0.        ]]


In [0]:
print(labels_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [34]:
# SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
print(X_train.shape)
print(labels_train.shape)
X_test = sc.transform(features_test)
from sklearn.utils import check_X_y
X, y = check_X_y(
            X_train, labels_train, multi_output=True)
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X,y)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='binary')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='binary')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='binary')
print('F-Measure: %.3f' % (score*100))

(672, 64)
(672, 64)


ValueError: ignored

In [35]:
# Kernel SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='binary')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='binary')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='binary')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [36]:
# Random Forest
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 8.036
Precision: 94.737
Recall: 8.036
F-Measure: 14.815


In [37]:
# Decision Tree
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 27.679
Precision: 27.679
Recall: 27.679
F-Measure: 27.679


In [38]:
# Naive Bayes
# Feature Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)
X, y = check_X_y(X_train, labels_train, accept_sparse=False, accept_large_sparse=True, dtype='numeric', order=None, copy=False, force_all_finite=True, ensure_2d=True, allow_nd=False, multi_output=True, ensure_min_samples=1, ensure_min_features=1, y_numeric=True, estimator=None)
# Fitting KernelSVM to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X, y)
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [39]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier =  KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 24.107
Precision: 76.056
Recall: 24.107
F-Measure: 36.610


In [40]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)
from sklearn.utils import check_X_y
X, y = check_X_y(
            X_train, labels_train, multi_output=True)
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, y)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored